In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Ensure these are installed and up-to-date
!pip install --upgrade pip
!pip install ultralytics opencv-python numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 93.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 156.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.4 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 45.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 66.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 80.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 39.8 MB/s eta 0:00:000

In [9]:
import cv2
from ultralytics import YOLO
import os

# --- Configuration ---
# Path to your input video file on Kaggle.
VIDEO_PATH = "/kaggle/input/re-mapping-dataset/15sec_input_720p.mp4"

# Path to your custom YOLOv11 model file on Kaggle.
MODEL_PATH = "/kaggle/input/object-detection-model/pytorch/default/1/best.pt"

# Path for the output video where results will be saved.
# This will save to the current working directory, typically /kaggle/working/ in Kaggle.
OUTPUT_VIDEO_PATH = "output.mp4"

# Minimum confidence threshold for detections to be considered.
# You might need to adjust this (e.g., lower for more detections, higher for fewer but more confident).
CONFIDENCE_THRESHOLD = 0.3

# Define the class name for players based on your model's output.
# Your model output confirmed: {0: 'ball', 1: 'goalkeeper', 2: 'player', 3: 'referee'}
PLAYER_CLASS_NAME = 'player'

# --- Step 1: Verify Input Files Exist ---
print(f"Checking for video file: {VIDEO_PATH}")
if not os.path.exists(VIDEO_PATH):
    print(f"Error: Video file not found at '{VIDEO_PATH}'")
    print("Please ensure the Kaggle dataset path is correct for the video.")
    exit()

print(f"Checking for model file: {MODEL_PATH}")
if not os.path.exists(MODEL_PATH):
    print(f"Error: Model file not found at '{MODEL_PATH}'")
    print("Please ensure the Kaggle dataset path is correct for the model.")
    exit()

# --- Step 2: Load the YOLOv11 Model ---
try:
    # Initialize YOLO model from your custom .pt file.
    model = YOLO(MODEL_PATH)
    print(f"YOLOv11 model loaded successfully from: {MODEL_PATH}")
    # Print the class names known by the model for confirmation (already confirmed, but good practice)
    print(f"Model class names: {model.names}")
except Exception as e:
    print(f"Error loading YOLOv11 model: {e}")
    print("Possible reasons: Corrupted model file, incorrect path, or incompatible Ultralytics version.")
    print("Try: pip install --upgrade ultralytics, and verify model file integrity.")
    exit()

# --- Step 3: Load the Input Video ---
cap = cv2.VideoCapture(VIDEO_PATH)

if not cap.isOpened():
    print(f"Error: Could not open video file '{VIDEO_PATH}'")
    print("Check if the video path is correct and if OpenCV has necessary codecs.")
    exit()

# Get video properties for output video creation
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

print(f"Input video details: Resolution={frame_width}x{frame_height}, FPS={fps}")

# --- Step 4: Prepare Output Video Writer ---
# Use 'mp4v' codec for .mp4 files.
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))

if not out.isOpened():
    print(f"Error: Could not create output video file '{OUTPUT_VIDEO_PATH}'")
    print("Ensure you have write permissions in the directory (e.g., /kaggle/working/).")
    exit() # Exit if we cannot save the output, as there's no display fallback

print(f"Output video will be saved to: {OUTPUT_VIDEO_PATH}")

# --- Step 5: Validate PLAYER_CLASS_NAME and get its ID ---
player_class_id = None
for k, v in model.names.items():
    if v == PLAYER_CLASS_NAME:
        player_class_id = k
        break

if player_class_id is None:
    print(f"Error: The configured PLAYER_CLASS_NAME '{PLAYER_CLASS_NAME}' was not found in the model's class names: {model.names.values()}")
    print("Please check the 'model.names' output and adjust PLAYER_CLASS_NAME if necessary.")
    exit()
else:
    print(f"Successfully identified player class: '{PLAYER_CLASS_NAME}' with ID: {player_class_id}")

# --- Step 6: Main Video Processing Loop with Tracking ---
print("\nStarting video processing with player re-identification (tracking)...")
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break # End of video or error reading frame

    frame_count += 1
    # Print progress every 30 frames (approx 1 second for 30 FPS video)
    if frame_count % 60 == 0:
        print(f"Processing frame {frame_count}...")

    # Perform tracking using model.track().
    # 'persist=True' ensures track IDs are maintained across frames.
    # 'tracker="bytetrack.yaml"' uses ByteTrack.
    # Pass the 'classes' argument to filter detections *before* tracking, improving efficiency.
    # We only want to track 'player' (and potentially 'goalkeeper' and 'referee' if treated as separate players)
    # Based on your requirement "identify each player", let's include 'goalkeeper' and 'referee' if they are
    # considered distinct "players" in this context. If only general field players, keep only 'player'.
    # For now, let's include 'goalkeeper' and 'referee' as well if they are distinct entities you want tracked.
    # Otherwise, just use `classes=[player_class_id]`.
    
    # Let's track 'player', 'goalkeeper', and 'referee' as they are all human figures.
    # Find IDs for 'goalkeeper' and 'referee'
    goalkeeper_class_id = None
    referee_class_id = None
    for k, v in model.names.items():
        if v == 'goalkeeper':
            goalkeeper_class_id = k
        elif v == 'referee':
            referee_class_id = k
    
    classes_to_track = [player_class_id]
    if goalkeeper_class_id is not None:
        classes_to_track.append(goalkeeper_class_id)
    if referee_class_id is not None:
        classes_to_track.append(referee_class_id)

    results = model.track(frame,
                          persist=True,
                          conf=CONFIDENCE_THRESHOLD,
                          verbose=False, # Suppress verbose output for each frame
                          tracker="bytetrack.yaml", # Default tracker configuration
                          classes=classes_to_track # Only process these specific classes for tracking
                         )

    # Process tracking results for the current frame
    for r in results:
        boxes = r.boxes # Access the Boxes object from the result
        if boxes.id is not None: # Check if tracking IDs are available (should be if tracking is working)
            for i, box in enumerate(boxes):
                x1, y1, x2, y2 = map(int, box.xyxy[0]) # Bounding box coordinates
                confidence = box.conf[0]             # Confidence score
                class_id = int(box.cls[0])           # Class ID
                track_id = int(boxes.id[i])          # Unique track ID assigned by the tracker
                class_name = model.names[class_id]   # Class name

                # Draw bounding box and ID for players, goalkeepers, and referees
                # Assign distinct colors if desired, or a single color for all human figures
                color = (0, 255, 0) # Default green for players
                if class_name == 'goalkeeper':
                    color = (255, 0, 0) # Blue for goalkeepers
                elif class_name == 'referee':
                    color = (0, 165, 255) # Orange for referees

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                # Text for ID and class
                text = f"{class_name} ID: {track_id}"
                cv2.putText(frame, text, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, lineType=cv2.LINE_AA)

                # Optional: Draw for 'ball' if you want to visualize it
                # if class_name == 'ball':
                #     cv2.circle(frame, (int((x1+x2)/2), int((y1+y2)/2)), 5, (0, 0, 255), -1) # Red circle for ball
                #     cv2.putText(frame, f"Ball", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, lineType=cv2.LINE_AA)


    # Write the processed frame to the output video file
    if out.isOpened():
        out.write(frame)

# --- Step 7: Release Resources ---
cap.release() # Release the input video object
if out.isOpened():
    out.release() # Release the output video writer object

print(f"\nVideo processing complete. Total frames processed: {frame_count}")
print(f"Output video saved successfully to: {OUTPUT_VIDEO_PATH}")
print("You can now download and play this video file to see the results.")

Checking for video file: /kaggle/input/re-mapping-dataset/15sec_input_720p.mp4
Checking for model file: /kaggle/input/object-detection-model/pytorch/default/1/best.pt
YOLOv11 model loaded successfully from: /kaggle/input/object-detection-model/pytorch/default/1/best.pt
Model class names: {0: 'ball', 1: 'goalkeeper', 2: 'player', 3: 'referee'}
Input video details: Resolution=1280x720, FPS=25
Output video will be saved to: output.mp4
Successfully identified player class: 'player' with ID: 2

Starting video processing with player re-identification (tracking)...
Processing frame 60...
Processing frame 120...
Processing frame 180...
Processing frame 240...
Processing frame 300...
Processing frame 360...

Video processing complete. Total frames processed: 375
Output video saved successfully to: output.mp4
You can now download and play this video file to see the results.
